## 1. Import Libraries

In [1]:
!pip install feature_engine

  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 68.9 MB/s eta 0:00:00
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


In [32]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 61.9 MB/s eta 0:00:00


In [36]:
import os
import pickle 

import boto3

import warnings

import pandas as pd
import numpy as np

import sklearn
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OrdinalEncoder,
    OneHotEncoder,
    MinMaxScaler,
    StandardScaler,
    PowerTransformer,
    FunctionTransformer
)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel

from feature_engine.outliers import Winsorizer
from feature_engine.encoding import (
    RareLabelEncoder,
    MeanEncoder,
    CountFrequencyEncoder    
)
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance

from sklearn.ensemble import RandomForestRegressor

import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)

import xgboost as xgb

## 2. Display settings

In [3]:
# for displaying all the columns 
pd.set_option("display.max_columns", None)

# for getting the output as a pandas dataframe and not as a numpy array from the scikitlearn transformers
sklearn.set_config(transform_output = "pandas")

warnings.filterwarnings('ignore')

## 3.Read Datasets

In [4]:
train = pd.read_csv("train.csv")
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,19:10:00,615,1.0,In-flight meal not included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1.0,In-flight meal not included,6540
2,Goair,2019-03-09,Banglore,New Delhi,11:40:00,14:35:00,175,0.0,No Info,7305
3,Air India,2019-06-12,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8366
4,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,07:40:00,525,1.0,In-flight meal not included,11087
...,...,...,...,...,...,...,...,...,...,...
6690,Jet Airways,2019-03-21,Delhi,Cochin,10:45:00,18:50:00,1925,2.0,No Info,11093
6691,Air India,2019-05-01,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8891
6692,Jet Airways,2019-06-01,Delhi,Cochin,14:00:00,19:00:00,300,1.0,In-flight meal not included,10262
6693,Air Asia,2019-06-24,Delhi,Cochin,07:55:00,13:25:00,330,1.0,No Info,6152


In [5]:
val = pd.read_csv("val.csv")
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No Info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1.0,No Info,9646
2,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,15:15:00,980,1.0,In-flight meal not included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1.0,No Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2.0,No Info,16704
...,...,...,...,...,...,...,...,...,...,...
1669,Spicejet,2019-05-01,Chennai,Kolkata,09:45:00,12:00:00,135,0.0,No Info,3597
1670,Indigo,2019-05-01,Kolkata,Banglore,08:10:00,13:00:00,290,1.0,No Info,5069
1671,Jet Airways,2019-05-27,Delhi,Cochin,05:30:00,12:35:00,425,2.0,In-flight meal not included,15544
1672,Jet Airways,2019-06-12,Mumbai,Hyderabad,19:35:00,21:05:00,90,0.0,In-flight meal not included,3210


In [6]:
test = pd.read_csv("test.csv")
test

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1.0,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0.0,No Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No Info,4804
...,...,...,...,...,...,...,...,...,...,...
2088,Jet Airways,2019-05-27,Delhi,Cochin,19:15:00,12:35:00,1040,1.0,In-flight meal not included,12898
2089,Multiple Carriers,2019-06-27,Delhi,Cochin,11:25:00,19:15:00,470,1.0,No Info,7155
2090,Jet Airways,2019-06-03,Delhi,Cochin,02:15:00,04:25:00,1570,1.0,In-flight meal not included,11627
2091,Multiple Carriers,2019-06-06,Delhi,Cochin,15:15:00,01:30:00,615,1.0,No Info,6795


## 4. Preprocessing Operations

In [7]:
# airline
air_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

#doj
feature_to_extract = ["month","week", "day_of_week", "day_of_year"]

doj_transformer = Pipeline(steps = [
    ("dt", DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format="mixed")),
    ("scale", MinMaxScaler())
])

# source and destination
location_pipe1 = Pipeline(steps = [
    ('grouper', RareLabelEncoder(tol= 0.1, replace_with="other", n_categories= 2 )),
    ('encoder', MeanEncoder()),
    ("scaler", PowerTransformer())
])

def is_north(X):
    columns = X.columns.to_list()
    north_cities = ["Delhi", "Kolkata", "Mumbai", "New Delhi"]
    return (
        X
        .assign(**{
            f"{col}_is_north" : X.loc[:, col].isin(north_cities).astype(int)
            for col in columns 
        })
        .drop(columns = columns)
    )

location_transformer = FeatureUnion(transformer_list= [
    ("part1", location_pipe1),
    ("part2", FunctionTransformer(func = is_north))
])

# dep_time and arrival_time
time_pipe1 = Pipeline(steps =[
    ("dt", DatetimeFeatures(features_to_extract=['hour', 'minute'])),
    ("scaler", MinMaxScaler())
])

def part_of_day(X, morning = 4, noon = 12, eve = 16, night = 20):
    columns = X.columns.to_list()
    X_temp = X.assign(**{
        col : pd.to_datetime(X.loc[:,col]).dt.hour
        for col in columns
    })
    
    return (
        X_temp
        .assign(**{
            f"{col}_part_of_day" : np.select(
                [X_temp.loc[:,col].between(morning, noon, inclusive = "left"),
                 X_temp.loc[:,col].between(noon, eve, inclusive = "left"),
                 X_temp.loc[:,col].between(eve, night, inclusive = "left")],
                ["morning", "afternoon", "evening"],
                default = "night"
            )
            for col in columns
        })
        .drop(columns = columns)
    )

time_pipe2 = Pipeline(steps = [
    ('part_of_day', FunctionTransformer(func = part_of_day)), 
    ('encoder', CountFrequencyEncoder()),
    ("scaler", MinMaxScaler())
])

time_transformer = FeatureUnion(transformer_list= [
    ('part1', time_pipe1),
    ("part2", time_pipe2)
])

# duration
def duration_categories(X):
    return (
        X
        .assign(duration_category = np.select(
                [X.loc[:,"duration"].between(0, 180, inclusive = "left"),
                X.loc[:,"duration"].between(180, 400, inclusive = "left")],
                ["short", "medium"],
                default = "long"
            )
        )
        .drop(columns = ["duration"])
    )

def is_over(X, value= 1000):
    return (
        X
        .assign(**{
            f"duration_over_{value}" : X.duration.ge(value).astype(int)
        })
        .drop(columns = "duration")
    )

class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    
    def __init__(self, variables = None, percentiles = [0.25, 0.5, 0.75], gamma = 0.1):
        self.variables = variables
        self.percentiles = percentiles
        self.gamma = gamma
    
    def fit(self, X, y = None):
        
        if not self.variables:
            self.variables = X.select_dtypes(include = "number").columns.to_list()
            
        self.reference_values_ = {
            col : (
                X
                .loc[:, col]
                .quantile(self.percentiles)
                .values
                .reshape(-1,1)
            )           
            for col in self.variables
        }
        
        return self
    
    def transform(self, X):
        objects = []
        
        for col in self.variables:
            columns = [f"{col}_rbf_{int(percentile * 100)}" for percentile in self.percentiles]
            obj = pd.DataFrame(
                data = rbf_kernel(X.loc[:, [col]], Y = self.reference_values_[col], gamma = self.gamma),
                columns = columns
            )
            objects.append(obj)
        
        return pd.concat(objects, axis = 1)

duration_pipe1 = Pipeline(steps=[
    ('rbf', RBFPercentileSimilarity()),
    ("scaler", PowerTransformer())
])

duration_pipe2 = Pipeline(steps = [
    ("cat", FunctionTransformer(func = duration_categories)),
    ("encoder", OrdinalEncoder(categories = [["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list = {
    ("part1", duration_pipe1),
    ("part2", duration_pipe2),
    ("part3", FunctionTransformer(func = is_over)),
    ("part4", StandardScaler())
})
 
duration_transformer = Pipeline(steps =[
    ("outliers", Winsorizer(capping_method = "iqr", fold = 1.5)),
    ("imputer", SimpleImputer(strategy = "median")),
    ("union", duration_union)
])

# total_stops
def is_direct(X):
    return(
        X
        .assign(
            is_direct_flight = X.total_stops.eq(0).astype(int)
        )
        .drop(columns = "total_stops")
    )

total_stops_transformer = FeatureUnion(transformer_list= [
    ("imputer", SimpleImputer(strategy= "most_frequent")),
    ("is_direct_", FunctionTransformer(func = is_direct))
])

# addtional_info
info_pipe1 = Pipeline(steps = [
    ("group", RareLabelEncoder(tol = 0.1, n_categories = 2, replace_with = "Other")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

def have_info(X):
    return(
        X
        .assign(
            additional_info = X.additional_info.ne("No Info").astype(int)
        )
        # .drop(columns = "a")
    )
    
info_union = FeatureUnion(transformer_list= [
    ("part1" , info_pipe1),
    ("part2", FunctionTransformer(func = have_info))
])

info_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy= "constant", fill_value="unknown")),
    ("union", info_union)
])

#Column Transformer
column_transfomer = ColumnTransformer(transformers =[
    ("air", air_transformer, ["airline"]),
    ("doj", doj_transformer, ["date_of_journey"]),
    ('location', location_transformer, ['source','destination']),
    ('time', time_transformer, ["dep_time", "arrival_time"]),
    ("dur", duration_transformer, ["duration"]),
    ("total_stops", total_stops_transformer, ["total_stops"]),
    ("info", info_transformer, ["additional_info"])
], remainder="passthrough")

# Feature Selection
estimator = RandomForestRegressor(n_estimators = 10, max_depth = 3, random_state=42)

selector = SelectBySingleFeaturePerformance(
    estimator = estimator,
    scoring = "r2",
    threshold=0.1
)

# Preprocessor
preprocessor = Pipeline(steps = [
    ("ct", column_transfomer),
    ("selector", selector)
])


In [8]:
preprocessor.fit(
    train.drop(columns = "price", axis = 1),
    train.price.copy()
)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('air',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('grouper',
                                                                   RareLabelEncoder(n_categories=2,
                                                                                    replace_with='Other',
                                                                                    tol=0.1)),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['airline']),
                                                 ('doj',
                                                  Pipeline(steps=[('dt',
                                                                   Datetime...
                                                                                                                                     replace_with='Other',
                                                                                                                                     tol=0.1)),
                                                                                                                   ('encoder',
                                                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                                                  sparse_output=False))])),
                                                                                                  ('part2',
                                                                                                   FunctionTransformer(func=<function have_info at 0x7fa2e30f4820>))]))]),
                                                  ['additional_info'])])),
                ('selector',
                 SelectBySingleFeaturePerformance(estimator=RandomForestRegressor(max_depth=3,
                                                                                  n_estimators=10,
                                                                                  random_state=42),
                                                  scoring='r2',
                                                  threshold=0.1))])

In [9]:
preprocessor.transform(train.drop(columns="price"))

,air__airline_Indigo,air__airline_Jet Airways,air__airline_Other,doj__date_of_journey_week,doj__date_of_journey_day_of_year,location__source,location__destination,dur__duration_category,dur__duration,dur__duration_rbf_25,dur__duration_over_1000,total_stops__total_stops,total_stops__is_direct_flight
0,0.0,1.0,0.0,0.176471,0.169492,-0.857930,-0.736484,2.0,-0.033916,-0.364262,0,1.0,0
1,0.0,1.0,0.0,0.235294,0.220339,1.065418,1.061694,2.0,0.046422,-0.364262,0,1.0,0
2,0.0,0.0,1.0,0.058824,0.067797,-0.857930,-0.736484,0.0,-0.917631,2.373008,0,0.0,1
3,0.0,0.0,0.0,0.882353,0.872881,-0.203928,-0.224351,2.0,-0.174507,-0.364262,0,1.0,0
4,0.0,1.0,0.0,0.117647,0.093220,-0.857930,-0.736484,2.0,-0.214676,-0.364262,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,0.0,1.0,0.0,0.176471,0.169492,1.065418,1.061694,2.0,2.597145,-0.364262,1,2.0,0
6691,0.0,0.0,0.0,0.529412,0.516949,-0.203928,-0.224351,2.0,-0.174507,-0.364262,0,1.0,0
6692,0.0,1.0,0.0,0.764706,0.779661,1.065418,1.061694,1.0,-0.666576,-0.364262,0,1.0,0
6693,0.0,0.0,1.0,1.000000,0.974576,1.065418,1.061694,1.0,-0.606322,-0.364262,0,1.0,0


## 5. Preprocess Data and Upload to Bucket

In [10]:
BUCKET_NAME = "sagemaker-flights-project.bucket"

DATA_PREFIX = "data"

In [11]:
def get_file_name(name):
    return f"{name}-pre.csv"

In [12]:
def export_data(data, name, pre):
    #Spliting the data into X and y subsets
    X = data.drop(columns = "price")
    y = data.price.copy()

    #transformation
    X_pre = pre.transform(X)

    #exporting
    file_name = get_file_name(name)

    #Sagemaker need the data in this formate like y should be first ana then the Xpre
    (
        y
        .to_frame()
        .join(X_pre)
        .to_csv(file_name, index =False, header = False)
    )

In [13]:
def upload_to_bucket(name):
    file_name = get_file_name(name)
    (
        boto3
        .Session()
        .resource("s3")
        .Bucket(BUCKET_NAME)
        .Object(os.path.join(DATA_PREFIX, f"{name}/{name}.csv"))
        .upload_file(file_name)
    )

In [14]:
def export_and_upload_to_bucket(data, name, pre):
    export_data(data, name, pre)
    upload_to_bucket(name)

In [15]:
export_and_upload_to_bucket(train, "train", preprocessor)

[04/22/25 20:25:57] INFO     Found credentials from IAM Role:                                   ]8;id=826341;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=296991;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [16]:
export_and_upload_to_bucket(val, "val", preprocessor)

[04/22/25 20:25:58] INFO     Found credentials from IAM Role:                                   ]8;id=442307;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=485791;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [17]:
export_and_upload_to_bucket(test, "test", preprocessor)

                    INFO     Found credentials from IAM Role:                                   ]8;id=271754;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=934876;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

## 6.Model Training and Hyperparameter Tuning

In [18]:
session = sagemaker.Session()
region_name = session.boto_region_name


[04/22/25 20:25:59] INFO     Found credentials from IAM Role:                                   ]8;id=687369;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=292290;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [19]:
output_path = f"s3://{BUCKET_NAME}/model/output"

In [20]:
model = Estimator(
    image_uri=sagemaker.image_uris.retrieve("xgboost", region_name, "1.2-1"),
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m4.xlarge",
    volume_size=5,
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600,
    sagemaker_session=session
)

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=637379;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=444432;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=881499;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=31825;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=921764;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=352195;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [41]:
model.set_hyperparameters(
    objective="reg:squarederror",
    num_round=100,
    eta=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    alpha=0.1
)

In [42]:
# hyperparameter_ranges = {
#     "eta": ContinuousParameter(0.05, 0.2),
#     "alpha": ContinuousParameter(0, 1),
#     "max_depth": IntegerParameter(3, 5)
# }

hyperparameter_ranges = {
    "eta": ContinuousParameter(0.01, 0.3),
    "alpha": ContinuousParameter(0, 2),
    "lambda": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(3, 10),
    "min_child_weight": IntegerParameter(1, 10),
    "subsample": ContinuousParameter(0.5, 1),
    "colsample_bytree": ContinuousParameter(0.5, 1),
}

In [23]:
tuner = HyperparameterTuner(
    estimator=model,
    objective_metric_name="validation:rmse",
    hyperparameter_ranges=hyperparameter_ranges,
    strategy="Bayesian",
    objective_type="Minimize"
)

## 6.Data Channels

In [24]:
def get_data_channel(name):
    bucket_path = f"s3://{BUCKET_NAME}/{DATA_PREFIX}/{name}"
    return TrainingInput(bucket_path, content_type="csv")

In [25]:
train_data_channel = get_data_channel("train")
train_data_channel

In [26]:
val_data_channel = get_data_channel("val")
val_data_channel

In [27]:
data_channels = {
    "train":train_data_channel,
    "validation":val_data_channel
}

## 8.Train and Tune Model

In [43]:
tuner.fit(data_channels)

[04/22/25 20:57:49] WARNING  No finished training job found associated with this estimator.       ]8;id=798117;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=387330;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  No finished training job found associated with this estimator.       ]8;id=93231;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=506588;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    INFO     Creating hyperparameter tuning job with name:                          ]8;id=159339;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=176254;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#3383\3383]8;;\
                             sagemaker-xgboost-250422-2057                                                         

......................................!


In [44]:
tuner.best_estimator()


2025-04-22 21:01:02 Starting - Preparing the instances for training
2025-04-22 21:01:02 Downloading - Downloading the training image
2025-04-22 21:01:02 Training - Training image download completed. Training in progress.
2025-04-22 21:01:02 Uploading - Uploading generated training model
2025-04-22 21:01:02 Completed - Training job completed


## 9.Model Evaluation

In [49]:
with open("xgboost-model-2", "rb") as f:
    best_model = pickle.load(f)

best_model

In [50]:
def evaluate_model(name):
    file_name = get_file_name(name)
    data = pd.read_csv(file_name)

    X = xgb.DMatrix(data.iloc[:,1:])
    y = data.iloc[:,0].copy()

    pred = best_model.predict(X)

    return r2_score(y, pred)

In [51]:
evaluate_model("train")

0.8346490859985352

In [52]:
evaluate_model("val")

0.7388879656791687

In [53]:
evaluate_model("test")

0.7575755715370178